In [1]:
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
import cv2
import xml.etree.ElementTree as ET
import matplotlib.image as mimp
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Reshape,LeakyReLU,MaxPooling2D,Dropout,Input, Flatten
from keras import backend as K
import tensorflow as tf
from tensorflow.train import AdamOptimizer
from tqdm import tqdm

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
# các param cần thiết cho quá trình training

img_w, img_h=640,480
grid_w,grid_h=16,12
nb_boxes=5
grid_size=40
width=int(img_w/grid_size)
height=int(img_h/grid_size)
B=5
coord_lamda=5
noobj_lamda=0.5
classes={'WBC':0,'RBC':1}
n_classes=len(classes)
anno_path='/Users/hit.flouxetine/datasets/BCCD_Dataset/BCCD/Annotations'
image_path='/Users/hit.flouxetine/datasets/BCCD_Dataset/BCCD/JPEGImages'

In [3]:
# lấy dữ liệu từ các file xml và decode chúng trành x_train, và y_train.
# y_train có hình dạng như sau:(width,heigt,n_classes+5*B)
def parse_annotation(anno_path,image_path):
    data=[]
    for each in sorted(os.listdir(anno_path)):
        image={}
        tree=ET.parse(anno_path+'/'+each)
        Object=[]
        for elem in tree.iter():
            if 'filename' in elem.tag:
                image['filename'] = image_path+'/' + elem.text
            if 'width' in elem.tag:
                image['width'] = int(elem.text)
            if 'height' in elem.tag:
                image['height'] = int(elem.text)
            if 'object' in elem.tag or 'part' in elem.tag:
                obj = {}
    
                for attr in list(elem):
                    if 'name' in attr.tag:
                        obj['name'] = attr.text
    
                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                obj['xmin'] = int(round(float(dim.text)))
                            if 'ymin' in dim.tag:
                                obj['ymin'] = int(round(float(dim.text)))
                            if 'xmax' in dim.tag:
                                obj['xmax'] = int(round(float(dim.text)))
                            if 'ymax' in dim.tag:
                                obj['ymax'] = int(round(float(dim.text)))
                Object.append(obj)
        image['object']=Object
        
        data.append(image)
    return data

In [4]:
def encode_data():
    data=parse_annotation(anno_path, image_path)
    N= len(data)
    out=[] #y_predict
    x=[] #chứa các ảnh
    for elem in data:
      
        img=mimp.imread(elem['filename'])
        x.append(img)
        truth=np.zeros((height,width,n_classes+B*5))
        cnt=np.zeros((height,width)) #to count how many object has been put in cell (i,j)

        for obj in elem['object']:
            
            bbox_x_cen, bbox_y_cen=(obj['xmax']-obj['xmin'])//2+obj['xmin'],(obj['ymax']-obj['ymin'])//2+obj['ymin']
            bbox_width,bbox_height= (obj['xmax']-obj['xmin']),(obj['ymax']-obj['ymin'])
        
        
        
            #possition of the bbox
            pos_x,pos_y=bbox_x_cen//grid_size,bbox_y_cen//grid_size
            cnt[pos_y][pos_x]+=1 # increase number of object in cell (i,j) one unit
        
            if obj['name']=='RBC':
                cell_classes=[0,1]
            else:
                cell_classes=[1,0]
        
            #normalize bbox
            bbox_x_cen,bbox_y_cen=bbox_x_cen/grid_size,bbox_y_cen/grid_size
            bbox_width,bbox_height=bbox_width/img_w, bbox_height/img_h
            bbox_c_score=1
        
            #assign value of bbox to the grib cell
            if cnt[pos_y][pos_x]==0:
                bbox=[bbox_x_cen,bbox_y_cen,bbox_width,bbox_height,bbox_c_score]*B+cell_classes
                truth[pos_y][pos_x]=bbox
                cnt[pos_y][pos_x]+=1
        
        out.append(truth)
    return [x,out]

In [5]:
x,y=encode_data()

- Chuyển x,y vừa tạo được thành numpy array với kiểu dữ liệu float32 

In [6]:
x=np.array(x,dtype='float32')
y=np.array(y,dtype='float32')

- Viết hàm tính IOU giữa các box. 
     - Input của hàm IOU sẽ là 2 tensor có hình dạng (batch_size,width,height,B,5)
     - Trong đó 5 giá trị của chiều cuối cùng lần lượt là x,y tại vị tri tâm của bounding box, chiều dài, và chiều cao của box

In [7]:
def IOU(a,b):
    '''
        input: two tensor with shape (batch, h, w, number_of_box, 5) represent each box in the grid cell
                each box has exactly 5 values which are x_center, y_center, width and height of it.
        output:a tensor is iou between those boxes
    '''
    interval_x=K.maximum(0.0,K.minimum(b[...,0]+b[...,2]/2,a[...,0]+a[...,2]/2)-K.maximum(b[...,0]-b[...,2]/2,a[...,0]-a[...,2]/2))
    interval_y=K.maximum(K.minimum(b[...,1]+b[...,3]/2,a[...,1]+a[...,3]/2)-K.maximum(b[...,1]-b[...,3]/2,a[...,1]-a[...,3]/2),0.0)
    
    bbox_s=a[...,2]*a[...,3]
    groundtruth_s=b[...,2]*b[...,3]
    intersect=interval_x*interval_y
    union=bbox_s+groundtruth_s-intersect
    
    return intersect/union

- Viết hàm custom loss cho yolo.
    - đầu vào bao gồm y_true(giá trị thực) và y_predict(giá trị mà model dự đoán ra.
    - shape(Batch_size,width,height,B*5+n_classes)
    - Lưu ý: Các toạ độ bounding box phải được đưa về toạ độ offset.

In [8]:
def custom_loss(y_true, y_pred):
    '''
        shape: (batch,height,width,nb_boxes*5+class)
    '''
    grid=[[float(x),float(y),float(0),float(0),float(0)]*nb_boxes for x in range(grid_h) for y in range(grid_w)]
    grid=K.variable(grid)
    grid=K.reshape(grid,(1,grid_h,grid_w,nb_boxes,5))
    
    y_pred_class=y_pred[...,B*5:]
    y_true_class=y_true[...,B*5:]
    
    y_pred_coord=y_pred[...,:B*5]
    y_true_coord=y_true[...,:B*5]
    
    y_pred_coord=K.reshape(y_pred_coord,(-1,grid_h,grid_w,nb_boxes,5))
    y_true_coord=K.reshape(y_true_coord,(-1,grid_h,grid_w,nb_boxes,5))
    
    y_pred_coord=y_pred_coord+grid
    
    #calculate IOU
    iou=IOU(y_true_coord,y_pred_coord)
    max_iou=tf.reduce_max(iou,axis=3,keepdims=True)
    delta=y_true_coord[...,-1]
    object_mask=tf.cast((max_iou<=iou), dtype=tf.float32)*delta
    noobject_mask=tf.cast((max_iou>iou), dtype=tf.float32)*delta
    
    #coordinate loss
    
    #x_center, y_center
    xy_pred_coord=y_pred_coord[...,:2]
    xy_true_coord=y_true_coord[...,:2]
    xy_score= coord_lamda*K.sum(K.sum(K.square(xy_pred_coord-xy_true_coord),axis=-1)*object_mask)
    #w,h coordinate
    wh_pred_coord=y_pred_coord[...,2:4]
    wh_true_coord=y_true_coord[...,2:4]
    wh_score= coord_lamda*K.sum(K.sum(K.square(K.sqrt(wh_pred_coord)-K.sqrt(wh_true_coord)),axis=-1)*object_mask)
    
    
    #confidence score
    
    c_pred_score=y_pred_coord[...,4]
    c_true_score=y_true_coord[...,4]
    
    c_score= K.sum(K.square(c_true_score*iou-c_pred_score)*object_mask)
    c_score+=noobj_lamda*K.sum(K.square(c_true_score*iou-c_pred_score)*noobject_mask)
    
    
    #class loss
    cell_mask=y_true[...,-1]
    class_score=K.sum(K.sum(K.square(y_true_class-y_pred_class),axis=-1)*cell_mask)
    
    return xy_score+wh_score+c_score+class_score

- khởi tạo model

In [9]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3), input_shape=(img_h,img_w,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters=64,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(3,3), strides=2))
    
model.add(Conv2D(filters=128, kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters=128,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))

model.add(Conv2D(filters=256, kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters=256,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))

model.add(Conv2D(filters=512, kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters=512,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters=512,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))

model.add(Conv2D(filters=512, kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters=512,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters=512,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))

model.add(Flatten())
model.add(Dense(1088))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1096))
model.add(LeakyReLU(0.1))
model.add(Dense(units=height*width*(n_classes+5*B), activation='sigmoid'))
model.add(Reshape((height,width,(n_classes+5*B))))

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 478, 638, 64)      1792      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 478, 638, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 476, 636, 64)      36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 476, 636, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 237, 317, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 235, 315, 128)     73856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 235, 315, 128)    

In [11]:
adam = AdamOptimizer(learning_rate=0.001)
model.compile(loss=custom_loss, optimizer=adam) # better

- Sau khi compile model xong thì chúng ta sẽ đi vào training.
- số lượng epoch, batch_size sẽ được khai báo bên dưới

In [ ]:
epochs = 100
batch_size = 64
N=x.shape[0]
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch+1,))
    for step in range(N//batch_size +1):
        index=step*batch_size
        last_index=min(index+batch_size,N)
        x_batch_train=K.variable(x[index:last_index])
        y_batch_train=K.variable(y[index:last_index])
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = custom_loss(y_batch_train, logits)
            loss_value += sum(model.losses)
            grads = tape.gradient(loss_value, model.trainable_weights)
        adam.apply_gradients(zip(grads, model.trainable_weights))
        
        print('Training loss (for one batch) at step %s' % (step))
        print('Seen so far: %s samples' % ((step + 1) * 64))

Start of epoch 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Training loss (for one batch) at step 0
Seen so far: 64 samples
Training loss (for one batch) at step 1
Seen so far: 128 samples
Training loss (for one batch) at step 2
Seen so far: 192 samples
Training loss (for one batch) at step 3
Seen so far: 256 samples
Training loss (for one batch) at step 4
Seen so far: 320 samples
Training loss (for one batch) at step 5
Seen so far: 384 samples
Start of epoch 2
